# Evaluate trained DeepParticleNet

In [ ]:
import os
import sys

# Add root directory to the python search path, if it is not already in there.
root_dir = os.path.abspath("..")
if root_dir not in sys.path:
    sys.path.append(root_dir)

# Import the DeepParticleNet modules.
from dpn.config import Config
from dpn.dataset import Dataset
from dpn.model import Model
from dpn.sizedistribution import SizeDistribution as PSD
from dpn import visualize

%matplotlib inline 

dataset_path = os.path.join(root_dir, "datasets")
dataset_name = "mpac"

## Setup configuration

In [ ]:
class MpacConfig(Config):

    BACKBONE = "resnet50"
    USE_PRETRAINED_WEIGHTS = "coco"

# Load previous configuration and make some adjustments.
config = Config.load(log_dir)

config.GPU_COUNT = 1
config.IMAGES_PER_GPU = 1
config.BATCH_SIZE = 1

# Minimum probability value to accept a detected instance
# ROIs below this threshold are skipped (default: 0.7)
config.DETECTION_MIN_CONFIDENCE = 0.5

# Non-maximum suppression threshold for detection (default: 0.3)
config.DETECTION_NMS_THRESHOLD = 0.3

## Load Model

In [ ]:
# Create model in inference mode
model = Model(mode="inference", 
              model_dir=model_dir,
              config=config)

In [ ]:
# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

## Load Test Dataset

In [ ]:
# Inherit from the Dataset class.
class MpacDataset(Dataset):
    MONOCLASS = "sphere"  # The dataset only has one class.

# Test dataset.
dataset_name = "test_s01"
dataset = MpacDataset()
dataset.load_dataset(dataset_path, dataset_name)
dataset.prepare()

## Run Detection

In [ ]:
results = model.analyze_dataset(dataset)

In [ ]:
results.display_detection_image(2)

## Compare PSDs

In [ ]:
results_gt = dataset.get_ground_truth()
evaluation_mode = "maximum_feret_diameter"
psd_gt = results_gt.to_size_distribution(evaluation_mode)

psd = results.to_size_distribution(evaluation_mode)

In [ ]:
psd.compare(psd_gt)

In [ ]:
figure_handle = visualize.plot_size_distributions([psd_gt,psd],
                                                  ["Manual","DPN"],
                                                  bins=np.linspace(0,150,31),
                                                  ncol=1
                                                 )